In [4]:
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score

# Load dataset
df = pd.read_csv("/Users/dana/Desktop/dl project/character_normalization_pairs_cleaned_output.csv")

# Split dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Load mBERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


# Define a custom Dataset class
class NormalizationDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item["input"]
        reference_text = item["reference"]

        # Tokenize input and reference text at the character level
        input_encoding = self.tokenizer(input_text, return_tensors='pt', padding='max_length', truncation=True, max_length=32)
        reference_encoding = self.tokenizer(reference_text, return_tensors='pt', padding='max_length', truncation=True, max_length=32)

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": reference_encoding["input_ids"].squeeze()
        }

# Prepare the dataset and dataloaders
train_dataset = NormalizationDataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

# Load pre-trained mBERT model
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./mbert-kazakh-normalizer",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Set up the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Start training
trainer.train()


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`